<h1>Segunda Questão</h1>

<p>O dataset utilizado foi o mesmo disponibilizado na primeira questão.</p>

<p>Link: https://www.dropbox.com/s/s2cyx82uxsv03rq/dados-ex5.txt?dl=0</p>


In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split, cross_val_score, validation_curve
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report,confusion_matrix
from math import sqrt

<h3>Leitura do dataset</h3>

In [ ]:
df = pd.read_csv('dados-ex5.txt') 
print(df.shape)
df.describe().transpose()

<h3>Definição da coluna alvo</h3>

In [ ]:
target_column = ['1.000000000000000000e+00'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

<h3>Separação do dataset em treino (80%) e teste (20%)</h3>

In [ ]:
X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

print(X_train.shape)
print(X_test.shape)

<h3>Definição do modelo da MLP</h3>

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(50,50,50,50,50,50,50), max_iter=100, alpha=1e-4, activation='relu',
                    solver='adam', verbose=10, random_state=1,
                    learning_rate_init=.1)

<h3>Projeção da curva de Aprendizado</h3>
<p>Relação entre o custo e o número de iterações</p>

In [ ]:
mlp.fit(X_train,y_train)
plt.ylabel('cost')
plt.xlabel('iterations')
plt.title("Learning rate =" + str(0.001))
plt.plot(mlp.loss_curve_)
plt.show()

<h3>Projeção da acurácia ao longo das épocas</h3>

In [ ]:
N_TRAIN_SAMPLES = X_train.shape[0]
N_EPOCHS = 60
N_BATCH = 128
N_CLASSES = np.unique(y_train)
scores_train = []
scores_test = []
epoch = 0

while epoch < N_EPOCHS:
    print('epoch: ', epoch)
    # SHUFFLING
    random_perm = np.random.permutation(X_train.shape[0])
    mini_batch_index = 0
    while True:
        # MINI-BATCH
        indices = random_perm[mini_batch_index:mini_batch_index + N_BATCH]
        mlp.partial_fit(X_train[indices], y_train[indices], classes=N_CLASSES)
        mini_batch_index += N_BATCH

        if mini_batch_index >= N_TRAIN_SAMPLES:
            break
            
    # SCORE TRAIN
    scores_train.append(mlp.score(X_train, y_train))

    # SCORE TEST
    scores_test.append(mlp.score(X_test, y_test))

    epoch += 1

fig, ax = plt.subplots(2, sharex=True, sharey=True)
ax[0].plot(scores_train)
ax[0].set_title('Train')
ax[1].plot(scores_test)
ax[1].set_title('Test')
fig.suptitle("Accuracy over epochs", fontsize=14)
plt.show()